# Convert raw to bmp image format

Copyright U. Raich 16.2.2026
This program is part of the IoT course at the University of Cape Coast, Ghana It is released under the MIT license


Get the filename and extract the width and height parameters from the filename

In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import re
import struct
import os


In [2]:
filename = '../Python41x49.raw'

In [3]:
m = re.search(r"\d", filename)

In [4]:
if m:
    print("Index of first digit: ",m.start())

Index of first digit:  9


In [5]:
tmp = filename[m.start():].split('.')[0]
print(tmp)

41x49


In [6]:
size = tmp.split('x')
width = int(size[0])
height = int(size[1])
print("width: {:d}, height: {:d}".format(width,height))

width: 41, height: 49


Now that we the width and height of the pixel image, let's read the file

In [7]:
f = open(filename,"rb")
pixels_raw_ba = bytearray(f.read())
f.close()

In [ ]:
print/"filename: ",file_name)

## Create the file header for the bmp file

In [ ]:
BI_BITFIELDS = 3

In [ ]:
colorMaskRed   = 0xf800
colorMaskGreen = 0x07e0
colorMaskBlue  = 0x001f
colorMask = struct.pack("III",colorMaskRed,colorMaskGreen,colorMaskBlue)
print(len(colorMask))
print(colorMask)

In [ ]:
BI_RGB = 0
infoHeader = bytearray(40)
infoHdrSize = len(infoHeader)
imageWidth = width
imageHeight = height
noOfPlanes = 1
noOfBitsPerPixel = 16
compression = BI_BITFIELDS              # the bit fields define rgb565 pixel format
imageSize = 0                           # this works for uncompressed images
imageSize = imageWidth * imageHeight * 2
hor_resolution = 0                      # no preference
ver_resolution = 0                      # no preference
noOfColors = 0                          # only used with color maps
importantColors = 0
infoHdr = struct.pack("3i2h6i",infoHdrSize,imageWidth,imageHeight,noOfPlanes,
                      noOfBitsPerPixel,compression,imageSize,
                      hor_resolution,ver_resolution,
                      noOfColors,importantColors)

In [ ]:
fileHdrSize = 14
magic = 'BM'.encode('ascii')
fileSize = fileHdrSize + len(infoHdr) + len(colorMask) + width*height*2
reserved = 0
offset = fileHdrSize + len(infoHdr) + len(colorMask)
hdr = magic + struct.pack('iHHi',fileSize,reserved,reserved,offset)
print("File header: ")
for i in range(len(hdr)-1):
    print("0x{:02x}, ".format(hdr[i]),end="")
print("0x{:02x}, ".format(hdr[i]))

swap bytes: colors are little endian

In [ ]:
pixels_swapped = bytearray(width*height*2)
for i in range(width*height):
    pixels_swapped[2*i] = pixels_raw_ba[2*i+1]
    pixels_swapped[2*i+1] = pixels_raw_ba[2*i]

rows are stored bottom up and padded to 4 byte alignment

In [ ]:
print(range(height-1,-1))

In [ ]:
upside_down = bytearray()
for col  in range(height-1,-1,-1):
    print(col*width*2,":",(col+1)*width*2)
    row = pixels_swapped[col*width*2: (col+1)*width*2]
    if len(row) % 4:
        row = row + bytes([0,0]) # pad to 4 byte boundary
    upside_down = upside_down + row

Open a binaryfile for writing
To get at the filename, split the filename string from the right and only at the first '.'

In [8]:
bmp_filename = filename.rsplit('.',1)[0] + ".bmp"
print("original filename: ",filename)
print("bmp filename: ",bmp_filename)

original filename:  ../Python41x49.raw
bmp filename:  ../Python41x49.bmp


In [ ]:
bmp_file = open(bmp_filename,"wb")
# write the header
bmp_file.write(hdr)
bmp_file.write(infoHdr)
bmp_file.write(colorMask)
bmp_file.write(upside_down)
bmp_file.close()